# Basic example

Multiplication of two integer numbers is pretty simple.
Additional information can be got from [documentation](https://docs.python.org/3/library/ctypes.html).

## Write C code

We want to multiply two integers and get integer as the result
```c
int mul(int a, int b) {
    return a * b;
}
```

Notice, that it's not mandatory to have `main` entry point function

## Compile shared library (dll in Windows)

```bash
gcc -shared basics.c -o lib_basics.so
```

## Load created library in Python code

In [5]:
from ctypes import cdll

basics = cdll.LoadLibrary('./lib_basics.so')

## Usage

Now it's pretty simple to call `mul` function just like it was a property of module `setters`

In [9]:
basics.mul(2, 5)

10

See? It's easy

# Change the long passed to your function

Long number in Python represented as an array of longs:
```
{d0, d1, d2, ...}
```

We can change each one of them, so why not?

## Prepare shared library

C function:
```c
#include <Python.h>

int set_long(PyLongObject* o, long new_value,

             size_t digit) {
             
    o->ob_digit[digit] = new_value;
    
    return 0;
}
```

File should be compiled to shared library (dll in Windows).

Makefile for Linux:
```
FLAGS=-shared
LIBRARIES=-I/usr/include/python3.4
BUILD_LIBRARY=gcc $(FLAGS) $(LIBRARIES)
all:
    $(BUILD_LIBRARY) setters.c -o lib_setters.so
```

## Use shared library

Load the library

In [8]:
from ctypes import cdll

setters = cdll.LoadLibrary('./lib_setters.so')

It's handy to create our own Python wrapper for this C function

In [2]:
from ctypes import c_long, c_size_t, c_voidp

def change_long(a, b=0, digit=0):
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Don't forget, that Python interpreter will not create new objects for little integers like `0`, so we should avoid assigning of new values to such numbers, because they will be changed in all places where they're used

In [3]:
from ctypes import c_long, c_size_t, c_voidp

def change_long(a, b=0, digit=0):
    args = (a, b, digit)
    if not all(type(a) is int for a in args):
        raise TypeError('All parameters should be of type "int", '
                        'but {} provided'.format(map(type, args)))
    if a + 0 is a:
        raise ValueError('No way. You don\'t want to break '
                         'your interpreter, right?')
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Let's recall, that we cannot change the value of integers inside of Python functions

In [4]:
def variable_info(text, variable):
    print('{:^30}: {:#05x} ({:#x})'.format(text, variable, id(variable)))

def foo(a, new_value):
    a = new_value

a = 2**10
variable_info('Before function call', a)
foo(a, 5)
variable_info('After function call', a)

     Before function call     : 0x400 (0x7fd100877f50)
     After function call      : 0x400 (0x7fd100877f50)


Now forget it and take a look on what we've done

In [5]:
a = 2**10
b = a
variable_info('Before function call', a)
change_long(a, 2, 0)
variable_info('After function call', a)
variable_info('What\'s about b? Here it is', b)

     Before function call     : 0x400 (0x7fd100877e30)
     After function call      : 0x002 (0x7fd100877e30)
  What's about b? Here it is  : 0x002 (0x7fd100877e30)


![hahaha](./lied.jpg)